# Introduction

This tutorial demonstrates basic steps involved in processing electron
backscatter diffraction (EBSD) patterns before crystal structure determination
(indexing) or other analysis.

For learning purposes, we recommend to use this notebook alongside our [user
guide](https://kikuchipy.org).

The data consists of 29 800 EBSD patterns from recrystallized polycrystalline
Nickel. It is available [via Zenodo](https://zenodo.org/record/3265037).

This functionality has been checked to run in kikuchipy-0.2.0 (May 2020). Bugs
are always possible, do not trust the code blindly, and if you experience any
issues, report them [in our issue
tracker](https://github.com/kikuchipy/kikuchipy-demos/issues).

Access all methods' documentation (docstrings) when the cursor is within the
function parantheses, like `function(CURSOR HERE)`, by holding `SHIFT` and
pressing `TAB`.

# Contents

1. [Loading and data inspection](#1)
2. [Background correction](#2)  
    1. [Static background correction](#2.1)  
    2. [Dynamic background correction](#2.2)
3. [Neighbour pattern averaging](#3)
4. [Further processing](#4)  
    1. [Intensity normalization](#4.1)  
    2. [Intensity rescaling/contrast stretching](#4.2)  
    3. [Filtering in the frequency domain (FFT filtering)](#4.3)  
    4. [Adaptive histogram equalization](#4.4)  

Set the interactive [Matplotlib backend](https://matplotlib.org/3.2.1/tutorials/introductory/usage.html#backends)
(`%matplotlib qt5`), import kikuchipy and other necessary libraries

In [1]:
%matplotlib qt5
import hyperspy.api as hs
import matplotlib.pyplot as plt
import numpy as np
import kikuchipy as kp

# 1. Loading and data inspection <a class="anchor" id="1"></a>

Load the EBSD data

In [2]:
data = "/Volumes/MBPe_15_Data/docs/school/Lehigh/jobs/NHI/working/EBSD/nickel_scan_gain/scan1_gain0db/Pattern.dat"
s = kp.load(data, lazy=False)  # lazy=True to not load into memory before calling s.compute()

Inspect the EBSD object `s`

In [3]:
s

<EBSD, title: Pattern, dimensions: (200, 149|60, 60)>

Inspect the data type

In [4]:
s.data.dtype

dtype('uint8')

Inspect the metadata associated with the EBSD object `s`

In [5]:
s.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EBSD
│       │       ├── azimuth_angle = 0.0
│       │       ├── binning = -1.0
│       │       ├── detector = NORDIF UF1100
│       │       ├── elevation_angle = 0.0
│       │       ├── exposure_time = 0.0035
│       │       ├── frame_number = -1
│       │       ├── frame_rate = 202
│       │       ├── gain = 0.0
│       │       ├── grid_type = square
│       │       ├── manufacturer = NORDIF
│       │       ├── sample_tilt = 70.0
│       │       ├── scan_time = 148
│       │       ├── static_background = array([[84, 87, 90, ..., 27, 29, 30],
       [87, 90, 93, ..., 27, 28, 30],
   ...  80, 82, ..., 28, 26, 26],
       [76, 78, 80, ..., 26, 26, 25]], dtype=uint8)
│       │       ├── version = 3.1.2
│       │       ├── xpc = -1.0
│       │       ├── ypc = -1.0
│       │       └── zpc = -1.0
│       ├── beam_energy = 20.0
│       ├── magnification = 200
│       ├── microscope = Hitachi SU-6600
│       └── worki

Set important experimental parameters, like the pattern centre (in EMsoft's convention)

In [6]:
s.set_experimental_parameters(
    xpc=38.86,
    ypc=132.65,
    zpc=16986.48,
    binning=8
)

See how this changed the metadata of the EBSD metadata node

In [7]:
ebsd_node = kp.signals.util.metadata_nodes("ebsd")

In [8]:
s.metadata.get_item(ebsd_node)

├── azimuth_angle = 0.0
├── binning = 8
├── detector = NORDIF UF1100
├── elevation_angle = 0.0
├── exposure_time = 0.0035
├── frame_number = -1
├── frame_rate = 202
├── gain = 0.0
├── grid_type = square
├── manufacturer = NORDIF
├── sample_tilt = 70.0
├── scan_time = 148
├── static_background = array([[84, 87, 90, ..., 27, 29, 30],
       [87, 90, 93, ..., 27, 28, 30],
   ...  80, 82, ..., 28, 26, 26],
       [76, 78, 80, ..., 26, 26, 25]], dtype=uint8)
├── version = 3.1.2
├── xpc = 38.86
├── ypc = 132.65
└── zpc = 16986.48

Set another metadata item and check that

In [9]:
s.metadata.set_item("General.title", "Recrystallized Ni")

In [10]:
s.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EBSD
│       │       ├── azimuth_angle = 0.0
│       │       ├── binning = 8
│       │       ├── detector = NORDIF UF1100
│       │       ├── elevation_angle = 0.0
│       │       ├── exposure_time = 0.0035
│       │       ├── frame_number = -1
│       │       ├── frame_rate = 202
│       │       ├── gain = 0.0
│       │       ├── grid_type = square
│       │       ├── manufacturer = NORDIF
│       │       ├── sample_tilt = 70.0
│       │       ├── scan_time = 148
│       │       ├── static_background = array([[84, 87, 90, ..., 27, 29, 30],
       [87, 90, 93, ..., 27, 28, 30],
   ...  80, 82, ..., 28, 26, 26],
       [76, 78, 80, ..., 26, 26, 25]], dtype=uint8)
│       │       ├── version = 3.1.2
│       │       ├── xpc = 38.86
│       │       ├── ypc = 132.65
│       │       └── zpc = 16986.48
│       ├── beam_energy = 20.0
│       ├── magnification = 200
│       ├── microscope = Hitachi SU-6600
│       └── w

In [11]:
s

<EBSD, title: Recrystallized Ni, dimensions: (200, 149|60, 60)>

Inspect the signal (detector) and navigation (sample) axes

In [12]:
s.axes_manager

<Axes manager, axes: (200, 149|60, 60)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    200 |      0 |       0 |     1.5 |     um 
               y |    149 |      0 |       0 |     1.5 |     um 
---------------- | ------ | ------ | ------- | ------- | ------ 
              dx |     60 |        |       0 |       1 |     um 
              dy |     60 |        |       0 |       1 |     um

Set the detector pixel size (and the detector origin to the detector centre)

In [13]:
s.set_detector_calibration(delta=70)  # Microns

Inspect the axes manager again (note the signal scale)

In [14]:
s.axes_manager

<Axes manager, axes: (200, 149|60, 60)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    200 |      0 |       0 |     1.5 |     um 
               y |    149 |      0 |       0 |     1.5 |     um 
---------------- | ------ | ------ | ------- | ------- | ------ 
              dx |     60 |        | -2.1e+03 |      70 |     um 
              dy |     60 |        | -2.1e+03 |      70 |     um

Plot the patterns

In [15]:
s.plot()

If we want to only operate on or inspect parts of a data set, e.g. a square
from an upper left pattern (row, col) = (10, 20) to a bottom right pattern
(row, col) = (50, 70)

In [16]:
#s = s.inav[10:50, 20:70]

# 2. Background correction <a class="anchor" id="2"></a>

## 2.A Static background correction <a class="anchor" id="2.1"></a>

Inspect the static background pattern

In [17]:
static_bg = s.metadata.get_item(ebsd_node + ".static_background")

In [18]:
plt.imshow(static_bg)

Remove the static background from each pattern by subtracting by it, but keeping relative intensities between the patterns

In [19]:
s.remove_static_background(operation="subtract", relative=True)

Removing the static background:
[########################################] | 100% Completed |  4.5s


Inspect the static background corrected patterns

In [20]:
s.plot()

## 2.B Dynamic background correction <a class="anchor" id="2.2"></a>

Before we remove the large scale variations on the detector (dynamic background), we want to inspect it. It can be obtained by Gaussian blurring in real or frequency space 

In [21]:
dynamic_bg = s.get_dynamic_background(
    filter_domain="frequency",  # or spatial
    std=8,
    truncate=4,
    dtype_out=np.float32,  # Takes up 4 times the RAM!
)

Getting the dynamic background:
[########################################] | 100% Completed | 11.2s


Inspect the dynamic background

In [22]:
dynamic_bg.plot()

Remove the dynamic background from each pattern by subtracting by it (of course loosing relative intensities between the patterns)

In [23]:
s.remove_dynamic_background(
    operation="subtract",
    filter_domain="frequency",
    std=8,
    truncate=4,
)

Removing the dynamic background:
[########################################] | 100% Completed | 14.8s


Inspect the dynamic background corrected patterns

In [24]:
s.plot()

# 3. Neighbour pattern averaging <a class="anchor" id="3"></a>

Create an averaging kernel

In [25]:
w_cross = kp.filters.Window(window="circular", shape=(3, 3))

Inspect the kernel and plot it

In [26]:
w_cross.plot()

(<Figure size 640x480 with 2 Axes>,
 <matplotlib.colorbar.Colorbar at 0x157a9fca0>)

In [27]:
w_cross

Window (3, 3) circular
[[0. 1. 0.]
 [1. 1. 1.]
 [0. 1. 0.]]

Create a Gaussian averaging kernel

In [28]:
w_gauss = kp.filters.Window(window="gaussian", std=1, shape=(9, 9))

Inspect it

In [29]:
w_gauss

Window (9, 9) gaussian
[[0.     0.     0.     0.0002 0.0003 0.0002 0.     0.     0.    ]
 [0.     0.0001 0.0015 0.0067 0.0111 0.0067 0.0015 0.0001 0.    ]
 [0.     0.0015 0.0183 0.0821 0.1353 0.0821 0.0183 0.0015 0.    ]
 [0.0002 0.0067 0.0821 0.3679 0.6065 0.3679 0.0821 0.0067 0.0002]
 [0.0003 0.0111 0.1353 0.6065 1.     0.6065 0.1353 0.0111 0.0003]
 [0.0002 0.0067 0.0821 0.3679 0.6065 0.3679 0.0821 0.0067 0.0002]
 [0.     0.0015 0.0183 0.0821 0.1353 0.0821 0.0183 0.0015 0.    ]
 [0.     0.0001 0.0015 0.0067 0.0111 0.0067 0.0015 0.0001 0.    ]
 [0.     0.     0.     0.0002 0.0003 0.0002 0.     0.     0.    ]]

Average each pattern (in the window centre, `1.`) with the neighbouring patterns with the window coefficients in `w_gauss` (by performing spatial correlation)

In [30]:
s.average_neighbour_patterns(window=w_gauss)

Averaging with the neighbour patterns:
[########################################] | 100% Completed | 15.5s


Inspect the averaged patterns

In [31]:
s.plot()

# 4. Further processing <a class="anchor" id="4"></a>

## 4.A Intensity normalization <a class="anchor" id="4.1"></a>

Normalize the patterns intensities to a mean of zero and a standard deviation of one

In [32]:
s.change_dtype(np.float32)  # Or passing dtype_out=np.float32 to normalize_intensity()

In [33]:
s.normalize_intensity()

Normalizing the image intensities:
[########################################] | 100% Completed |  2.6s


Inspect the normalized patterns

In [34]:
s.plot()

## 4.B Intensity rescaling/contrast stretching <a class="anchor" id="4.2"></a>

Rescale pattern intensities to fill the entire data type range of the `uint16` data type

In [35]:
s.rescale_intensity(dtype_out=np.uint16)

Rescaling the image intensities:
[########################################] | 100% Completed |  2.8s


Inspect the rescaled patterns

In [36]:
s.plot()

Perform contrast stretching on a copy of the data

In [37]:
s2 = s.deepcopy()  # Double the memory!

In [38]:
s2.rescale_intensity(percentiles=(0.5, 99.5))

Rescaling the image intensities:
[########################################] | 100% Completed | 10.7s


Compare the patterns before and after contrast stretching

In [39]:
hs.plot.plot_signals([s, s2])

## 4.C Filtering in the frequency domain <a class="anchor" id="4.3"></a>

Create a Gaussian lowpass transfer function of pattern shape to remove high frequency components (noise) in the patterns

In [40]:
pattern_shape = s.axes_manager.signal_shape

In [41]:
pattern_shape

(60, 60)

In [42]:
w_low = kp.filters.Window(
    window="lowpass",
    cutoff=22,
    cutoff_width=10,
    shape=pattern_shape
)

Inspect the filter

In [43]:
plt.figure()
plt.imshow(w_low)

Create a Gaussian highpass transfer function of pattern shape to remove low frequency components (large scale variations,
a variant of dynamic background correction) in the pattern

In [44]:
w_high = kp.filters.Window(
    window="highpass",
    cutoff=3,
    cutoff_width=2,
    shape=pattern_shape
)

Inspect the filter

In [45]:
plt.figure()
plt.imshow(w_high)

Create a combined Gaussian high and low pass filter

In [46]:
w = w_low * w_high

Inspect the combined filter

In [47]:
plt.figure()
plt.imshow(w)

Apply the filter in the frequency domain on a copy of the data

In [48]:
s3 = s.deepcopy()

s3.fft_filter(
    transfer_function=w,
    function_domain="frequency",  # It is already a transfer function
    shift=True,
)

FFT filtering:
[########################################] | 100% Completed | 16.8s


Compare the patterns before and after FFT filtering

In [49]:
hs.plot.plot_signals([s, s3])

## 4.D Adaptive histogram equalization <a class="anchor" id="4.4"></a>

Perform adaptive histogram equalization (like done in EMsoft's dictionary indexing program `EMEBSDDI`) on a copy of the data

In [50]:
s4 = s.deepcopy()

In [51]:
s4.adaptive_histogram_equalization()

Adaptive histogram equalization:
[########################################] | 100% Completed |  3min 35.7s


Plot the two data sets before and after equalization side-by-side

In [52]:
hs.plot.plot_signals([s, s4])

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/matplotlib/cbook/__init__.py", line 196, in process
    func(*args, **kwargs)
  File "/usr/local/lib/python3.8/site-packages/hyperspy/drawing/utils.py", line 172, in function_wrapper
    function()
  File "/usr/local/lib/python3.8/site-packages/hyperspy/drawing/figure.py", line 107, in _on_close
    self.events.closed.trigger(obj=self)
  File "<string>", line 4, in trigger
  File "/usr/local/lib/python3.8/site-packages/hyperspy/events.py", line 402, in trigger
    function(**{kw: kwargs.get(kw, None) for kw in kwsl})
  File "/usr/local/lib/python3.8/site-packages/hyperspy/signal.py", line 2133, in <lambda>
    lambda: self.events.data_changed.disconnect(self.update_plot),
  File "/usr/local/lib/python3.8/site-packages/hyperspy/events.py", line 372, in disconnect
    raise ValueError("The %s function is not connected to %s." %
ValueError: The <bound method BaseSignal.update_plot of <EBSD, title: Recrystall